In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp
import numpy as np


In [2]:
solver = pywraplp.Solver.CreateSolver('CBC')

## Variables de decisión

In [3]:
variables = {}

for i in range(1,6):
    for j in range(1, 6):
        for k in range(1,26):
            variables[(i,j,k)] = solver.BoolVar(f'X_{i},{j},{k}')

# variables

## Restricciones

In [4]:
# Cada número en la cuadricula tiene asociado solo un número de los disponibles
for i in range(1,6):
    for j in range(1,6):
        solver.Add(solver.Sum([variables[i,j,k] for k in range(1,26)]) == 1)

# Cada número disponible tiene que ir en un solo casillero de la cuadrícula
for k in range(1,26):
    k_variables = []
    for i in range(1,6):
        for j in range(1,6):
            k_variables.append(variables[i,j,k])
    solver.Add(solver.Sum([x for x in k_variables])== 1)


# Restricción de suma horizontal y vertical

for i in range(1,6):
    row = []
    for j in range(1,6):
        for k in range(1,26):
            row.append((variables[i,j,k], k))
    
    # Compara fila con columna
    for j in range(1,6):
        col = []
        for i2 in range(1,6):
            for k in range(1,26):
                col.append((variables[i2,j,k], k))
    
        solver.Add(solver.Sum([r[0]*r[1] for r in row]) == solver.Sum(c[0]*c[1] for c in col))


first_row = []
for j in range(1,6):
    for k in range(1,26):
        first_row.append((variables[1,j,k], k))
    
diag1 = []
diag2 = []
for j in range(1,6):
    for k in range(1,26):
        diag1.append((variables[j,j,k], k))
        diag2.append((variables[6-j,j,k], k))

solver.Add(solver.Sum([r[0]*r[1] for r in first_row]) == solver.Sum(c[0]*c[1] for c in diag1))
solver.Add(solver.Sum([r[0]*r[1] for r in first_row]) == solver.Sum(c[0]*c[1] for c in diag2))



<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fd4f4a6cea0> >

## Objective Function

In [5]:
solver.Maximize(solver.Sum([x for x in variables.values()]))

In [6]:
status = solver.Solve()

In [ ]:
matrix = pd.DataFrame(
    columns=[i for i in range(1,6)],
    index=[j for j in range(1,6)],
    data=0
)

In [ ]:
if status == pywraplp.Solver.OPTIMAL:
    print(f"Cantidad de valores = {solver.Objective().Value()}")
    for i in range(1,6):
        for j in range(1, 6):
            for k in range(1,26):
                if variables[i,j,k].solution_value() == 1:
                    matrix.loc[i,j] = k
    for i, row in matrix.iterrows():
        print(np.sum(row.values))
else:
    print("El problema no tiene una solución óptima.")

In [ ]:
matrix